In [35]:
import csv
import nltk
import string
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
#read csv
train_csv = pd.read_csv("../en_project_csv/train.csv")
test_csv  = pd.read_csv("../en_project_csv/test.csv")

train_csv = pd.DataFrame(train_csv)
test_csv  = pd.DataFrame(test_csv)

train_csv.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_csv['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
train_csv.fillna('', inplace=True)

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def remove_punctuation(text):
    punctuation_free = "".join([i for i in text if i not in string.punctuation])
    return punctuation_free


def tokenize(string):
    tokens = string.split()
    return tokens


train_csv['text']= train_csv['text'].apply(lambda x:remove_punctuation(x))
train_csv['text']= train_csv['text'].apply(lambda x: tokenize(x))

In [14]:
train_csv.head()

,id,keyword,location,text,target
0,1,,,"[Our, Deeds, are, the, Reason, of, this, earth...",1
1,4,,,"[Forest, fire, near, La, Ronge, Sask, Canada]",1
2,5,,,"[All, residents, asked, to, shelter, in, place...",1
3,6,,,"[13000, people, receive, wildfires, evacuation...",1
4,7,,,"[Just, got, sent, this, photo, from, Ruby, Ala...",1


In [15]:
train_csv['keyword'] = train_csv['keyword'].apply(lambda x: tokenize(x))
train_csv.tail()

,id,keyword,location,text,target
7608,10869,[],,"[Two, giant, cranes, holding, a, bridge, colla...",1
7609,10870,[],,"[ariaahrary, TheTawniest, The, out, of, contro...",1
7610,10871,[],,"[M194, 0104, UTC5km, S, of, Volcano, Hawaii, h...",1
7611,10872,[],,"[Police, investigating, after, an, ebike, coll...",1
7612,10873,[],,"[The, Latest, More, Homes, Razed, by, Northern...",1


In [16]:
train_csv.drop(columns = ['id'], inplace=True)

In [19]:
#remove stop words
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [20]:
def remove_stopwords(text):
    output = [i for i in text if i not in stopwords]
    return output

train_csv['text'] = train_csv['text'].apply(lambda x:remove_stopwords(x))

In [22]:
porter_stemmer = PorterStemmer()

In [23]:
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

train_csv['text'] = train_csv['text'].apply(lambda x: stemming(x))

In [25]:
wordnet_lemmatizer = WordNetLemmatizer()

In [26]:
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

In [27]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [28]:
#['a','b'] + ['c'] = ['a','b','c']

vocab = []

for i in train_csv['text'].values:
    vocab = vocab + i

print(len(vocab))

82531


In [29]:
set_vocab = set(vocab)
vocab = list(set_vocab)

print(len(vocab), type(vocab))

19822 <class 'list'>


In [30]:
train_csv['text_strings'] = train_csv['text'].apply(lambda x: ' '.join([str(elem) for elem in x]))
train_csv['text_strings'].head()

0        our deed reason earthquak may allah forgiv us
1                 forest fire near La rong sask canada
2    all resid ask shelter place notifi offic No ev...
3    13000 peopl receiv wildfir evacu order california
4    just got sent photo rubi alaska smoke wildfir ...
Name: text_strings, dtype: object

In [33]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_csv['text_strings'])

x_train = X.toarray()
x_train = np.array(x_train)

y_train = train_csv['target']

x_train.shape
y_train.shape

(7613,)

In [36]:
model_test = LogisticRegression(random_state=42)
model_test.fit(x_train,y_train)

model_predict = model_test.predict(x_train)
accuracy_score(y_train, model_predict)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9630894522527256

In [37]:
test_csv.fillna('', inplace=True)

In [38]:
test_csv.drop(columns=['id','keyword','location'], inplace=True)

In [39]:
test_csv['text'] = test_csv['text'].apply(lambda x:remove_punctuation(x))
test_csv['text'] = test_csv['text'].apply(lambda x: tokenize(x))
test_csv['text'] = test_csv['text'].apply(lambda x:remove_stopwords(x))
test_csv['text'] = test_csv['text'].apply(lambda x: stemming(x))

In [40]:
test_csv['text_strings'] = test_csv['text'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [41]:
x_test = vectorizer.transform(test_csv['text_strings'])

x_test = x_test.toarray()
x_test = np.array(x_test)

y_test_model_predict = model_test.predict(x_test)
y_test_model_predict

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [43]:
sub_csv = pd.read_csv("../en_project_csv/test.csv")
sub_csv['target'] = y_test_model_predict

final_sub = sub_csv[['id','target']]
final_sub.to_csv('final_submission.csv', index=False)

In [44]:
print(final_sub)

         id  target
0         0       1
1         2       1
2         3       1
3         9       1
4        11       1
...     ...     ...
3258  10861       1
3259  10865       1
3260  10868       1
3261  10874       1
3262  10875       0

[3263 rows x 2 columns]
